In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import LongformerTokenizer, LongformerForSequenceClassification, Trainer, TrainingArguments, EarlyStoppingCallback
import torch
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.utils.class_weight import compute_class_weight

In [2]:
# Load the dataset
df = pd.read_csv("combined_output.csv", on_bad_lines='skip')

In [3]:
df.head()

,original_id,source_name,title,url,body,date_published,language,date_modified,author_list,images,description,sentiment,emotions,entities,quotations,prValues,clipping,label,category
0,'5404c9d2fd24852afa122f2cc01cb3acba3c5d05b682d...,'kabargayo.com,'Kredit sepeda motor bisa terbayar jika Anda m...,'https://www.kabargayo.com/2024/09/19/kredit-s...,"'Jakarta, VIVA – Pembayaran kredit sepeda moto...",19/09/2024 22.32,'id,19/09/2024 22.32,'Aldi Hadad,'https://i1.wp.com/thumb.viva.co.id/media/fron...,NaN,'positive,NaN,"'Hari Pembayaran Berbayar atau Harcilnas 2024,...",NaN,5250000,Adira,NaN,NaN
1,'e1e3f8d68b58568e8217b7562d48de634fceb0d837135...,'viva.co.id,'Kredit Motor Bisa Lunas Jika Bayar Cicilan Te...,'https://www.viva.co.id/otomotif/tips/1753596-...,"'Jakarta, VIVA – Cicilan kredit motor yang ser...",19/09/2024 22.30,'id,19/09/2024 22.30,'Krisna Wicaksono,'https://thumb.viva.co.id/media/frontend/thumb...,NaN,'positive,NaN,"'Harinya Cicilan Lunas,2024,PT Adira Dinamika ...",NaN,5250000,Adira,NaN,NaN
2,'dca74b8fa4eabf60cebfa7b811ecb385872a0fd301eaf...,'kabarmegapolitan.pikiran-rakyat.com,'Adira Finance Umumkan Pemenang HARCILNAS 2024...,'https://kabarmegapolitan.pikiran-rakyat.com/b...,'KABARMEGAPOLITAN.com - PT Adira Dinamika Mult...,19/09/2024 21.45,'id,19/09/2024 21.45,'Yuliansyah,'https://assets.pikiran-rakyat.com/www/network...,NaN,'positive,'HARCILNAS merupakan wujud apresiasi kami kepa...,"'PT Adira Dinamika,Cicilan Lunas HARCILNAS,12 ...","(Person :Tania Endah Budhi ,Quote : HARCILNAS ...",5250000,Adira,NaN,NaN
3,'56c73e2a6d254a17a5cc21dee7ed0b4660c3af70c093b...,'banggairaya.id,"'Dapatkan Promo Menarik, Yamaha Prima Motor Ra...",'https://banggairaya.id/dapatkan-promo-menarik...,'BANGGAI RAYA- Yamaha Prima Motor ramaikan pam...,19/09/2024 19.45,'id,19/09/2024 19.45,'Chikal Connect,'https://i0.wp.com/banggairaya.id/wp-content/u...,NaN,'neutral,NaN,"'RAYA- Yamaha Prima Motor,Banggai Goverment Ex...",NaN,5250000,Adira,NaN,NaN
4,'1cd6c6db60224b6ee5f49cd5d6c62cd2850d9f3255721...,'jakarta.tribunnews.com,"'Sindikat Penipuan Leasing, Satu Bulan Ajukan ...",'https://jakarta.tribunnews.com/2024/09/19/sin...,'Laporan wartawan TribunJakarta.com Yusuf Bach...,19/09/2024 18.43,'id,19/09/2024 18.43,'Yusuf Bachtiar,'https://asset-2.tstatic.net/jakarta/foto/bank...,NaN,'neutral,'Pelaku ini melakukan pembiayaan pembelian ken...,"'Yusuf Bachtiar TRIBUNJAKARTACOM,MEDAN,SATRIA,...","(Person :Dedi ,Quote : Pelaku ini melakukan pe...",5250000,Adira,NaN,NaN


In [3]:
# Clean extra characters from all columns in the DataFrame
for column in df.columns:
    if df[column].dtype == 'object':  # Check if the column is of string type
        df[column] = df[column].str.strip("'")  # Remove extra characters

# Filter relevant columns
df_filtered = df[['body', 'sentiment']].dropna()


In [5]:
df.head()

,original_id,source_name,title,url,body,date_published,language,date_modified,author_list,images,description,sentiment,emotions,entities,quotations,prValues,clipping,label,category
0,5404c9d2fd24852afa122f2cc01cb3acba3c5d05b682d4...,kabargayo.com,Kredit sepeda motor bisa terbayar jika Anda me...,https://www.kabargayo.com/2024/09/19/kredit-se...,"Jakarta, VIVA – Pembayaran kredit sepeda motor...",19/09/2024 22.32,id,19/09/2024 22.32,Aldi Hadad,https://i1.wp.com/thumb.viva.co.id/media/front...,NaN,positive,NaN,"Hari Pembayaran Berbayar atau Harcilnas 2024,P...",NaN,5250000,Adira,NaN,NaN
1,e1e3f8d68b58568e8217b7562d48de634fceb0d8371356...,viva.co.id,Kredit Motor Bisa Lunas Jika Bayar Cicilan Tep...,https://www.viva.co.id/otomotif/tips/1753596-k...,"Jakarta, VIVA – Cicilan kredit motor yang seri...",19/09/2024 22.30,id,19/09/2024 22.30,Krisna Wicaksono,https://thumb.viva.co.id/media/frontend/thumbs...,NaN,positive,NaN,"Harinya Cicilan Lunas,2024,PT Adira Dinamika M...",NaN,5250000,Adira,NaN,NaN
2,dca74b8fa4eabf60cebfa7b811ecb385872a0fd301eaf5...,kabarmegapolitan.pikiran-rakyat.com,Adira Finance Umumkan Pemenang HARCILNAS 2024:...,https://kabarmegapolitan.pikiran-rakyat.com/bi...,KABARMEGAPOLITAN.com - PT Adira Dinamika Multi...,19/09/2024 21.45,id,19/09/2024 21.45,Yuliansyah,https://assets.pikiran-rakyat.com/www/network/...,NaN,positive,HARCILNAS merupakan wujud apresiasi kami kepad...,"PT Adira Dinamika,Cicilan Lunas HARCILNAS,12 p...","(Person :Tania Endah Budhi ,Quote : HARCILNAS ...",5250000,Adira,NaN,NaN
3,56c73e2a6d254a17a5cc21dee7ed0b4660c3af70c093b2...,banggairaya.id,"Dapatkan Promo Menarik, Yamaha Prima Motor Ram...",https://banggairaya.id/dapatkan-promo-menarik-...,BANGGAI RAYA- Yamaha Prima Motor ramaikan pame...,19/09/2024 19.45,id,19/09/2024 19.45,Chikal Connect,https://i0.wp.com/banggairaya.id/wp-content/up...,NaN,neutral,NaN,"RAYA- Yamaha Prima Motor,Banggai Goverment Exp...",NaN,5250000,Adira,NaN,NaN
4,1cd6c6db60224b6ee5f49cd5d6c62cd2850d9f3255721d...,jakarta.tribunnews.com,"Sindikat Penipuan Leasing, Satu Bulan Ajukan K...",https://jakarta.tribunnews.com/2024/09/19/sind...,Laporan wartawan TribunJakarta.com Yusuf Bacht...,19/09/2024 18.43,id,19/09/2024 18.43,Yusuf Bachtiar,https://asset-2.tstatic.net/jakarta/foto/bank/...,NaN,neutral,Pelaku ini melakukan pembiayaan pembelian kend...,"Yusuf Bachtiar TRIBUNJAKARTACOM,MEDAN,SATRIA,T...","(Person :Dedi ,Quote : Pelaku ini melakukan pe...",5250000,Adira,NaN,NaN


In [4]:
# Balance dataset classes
class_counts = df_filtered['sentiment'].value_counts()
min_class = class_counts.min()
df_balanced = df_filtered.groupby('sentiment').apply(lambda x: x.sample(min_class)).reset_index(drop=True)

In [5]:
# Ensure balanced classes during split
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df_filtered['body'].tolist(), df_filtered['sentiment'].tolist(),
    test_size=0.2, random_state=42, stratify=df_filtered['sentiment']
)

In [8]:
from transformers import LongformerTokenizer

# Load tokenizer and tokenize texts
tokenizer = LongformerTokenizer.from_pretrained("allenai/longformer-base-4096")
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=512)
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=512)


In [6]:
# Map sentiment labels to integers
label_mapping = {'positive': 2, 'neutral': 1, 'negative': 0}
train_labels = [label_mapping[label] for label in train_labels]
val_labels = [label_mapping[label] for label in val_labels]

In [10]:
class SentimentDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)


In [10]:
# Create datasets
train_dataset = SentimentDataset(train_encodings, train_labels)
val_dataset = SentimentDataset(val_encodings, val_labels)

NameError: name 'SentimentDataset' is not defined

: 

In [12]:
# Check if GPU is available
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

Using device: cuda


In [13]:
# Load Longformer model for sequence classification
model = LongformerForSequenceClassification.from_pretrained("allenai/longformer-base-4096", num_labels=3)

Some weights of LongformerForSequenceClassification were not initialized from the model checkpoint at allenai/longformer-base-4096 and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
class_weights = compute_class_weight('balanced', classes=np.unique(train_labels), y=train_labels)
class_weights = torch.tensor(class_weights, dtype=torch.float).to(device)

# Adjust model's loss function to include class weights
model.config.problem_type = "single_label_classification"
model.config.class_weights = class_weights.tolist()

In [15]:
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=4,  # Increased for efficiency
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=2,  # Simulates larger batch size
    num_train_epochs=4,  # Longer training
    learning_rate=2e-5,  # Lower for stability
    weight_decay=0.01,
    fp16=True,  # Mixed precision
    logging_dir='./logs',
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model='eval_loss',
    greater_is_better=False,
    lr_scheduler_type="reduce_lr_on_plateau",
    save_total_limit=2,
)

c:\Users\rache\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [16]:

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=10)],  # Increase patience
)

In [17]:
import torch
torch.cuda.empty_cache()


In [18]:
# Train the model
trainer.train()

  0%|          | 10/3600 [00:09<50:23,  1.19it/s] 

{'loss': 1.0461, 'grad_norm': 5.252406597137451, 'learning_rate': 2e-05, 'epoch': 0.01}


  1%|          | 20/3600 [00:17<49:35,  1.20it/s]

{'loss': 1.0993, 'grad_norm': 5.004133224487305, 'learning_rate': 2e-05, 'epoch': 0.02}


  1%|          | 30/3600 [00:25<49:29,  1.20it/s]

...

                                                   
100%|██████████| 3600/3600 [53:51<00:00,  1.20it/s]

{'eval_loss': 0.7218886017799377, 'eval_runtime': 58.5336, 'eval_samples_per_second': 30.769, 'eval_steps_per_second': 7.705, 'epoch': 4.0}


100%|██████████| 3600/3600 [53:53<00:00,  1.11it/s]

{'train_runtime': 3233.7458, 'train_samples_per_second': 8.906, 'train_steps_per_second': 1.113, 'train_loss': 0.633773207200898, 'epoch': 4.0}


TrainOutput(global_step=3600, training_loss=0.633773207200898, metrics={'train_runtime': 3233.7458, 'train_samples_per_second': 8.906, 'train_steps_per_second': 1.113, 'total_flos': 9458731243929600.0, 'train_loss': 0.633773207200898, 'epoch': 4.0})

In [19]:
print(f"Number of training examples: {len(train_dataset)}")
print(f"Batch size: {training_args.per_device_train_batch_size}")


Number of training examples: 7200
Batch size: 4


In [20]:
# Save the trained model and tokenizer
model.save_pretrained('./longformer_model4')
tokenizer.save_pretrained('./longformer_model4')

('./longformer_model4\\tokenizer_config.json',
 './longformer_model4\\special_tokens_map.json',
 './longformer_model4\\vocab.json',
 './longformer_model4\\merges.txt',
 './longformer_model4\\added_tokens.json')

In [21]:
# Evaluate the model
results = trainer.evaluate()

100%|██████████| 451/451 [00:57<00:00,  7.80it/s]


In [22]:
print(results)

{'eval_loss': 0.6839324831962585, 'eval_runtime': 58.0223, 'eval_samples_per_second': 31.04, 'eval_steps_per_second': 7.773, 'epoch': 4.0}


In [39]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report

# After training, get predictions on the validation dataset
predictions = trainer.predict(val_dataset)

# Get the predicted logits
logits = predictions.predictions

# Convert logits to predicted class labels
predicted_labels = np.argmax(logits, axis=1)

# Get the true labels
true_labels = val_labels

# Calculate evaluation metrics
accuracy = accuracy_score(true_labels, predicted_labels)
f1 = f1_score(true_labels, predicted_labels, average='weighted')
precision = precision_score(true_labels, predicted_labels, average='weighted')
recall = recall_score(true_labels, predicted_labels, average='weighted')

# Display metrics
print(f"Accuracy: {accuracy:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")

# Detailed classification report
print("\nClassification Report:")
print(classification_report(true_labels, predicted_labels))


100%|██████████| 451/451 [00:59<00:00,  7.61it/s]

Accuracy: 0.7529
F1 Score: 0.7496
Precision: 0.7592
Recall: 0.7529

Classification Report:
              precision    recall  f1-score   support

           0       0.72      0.63      0.68       338
           1       0.73      0.87      0.79       836
           2       0.82      0.66      0.73       627

    accuracy                           0.75      1801
   macro avg       0.76      0.72      0.73      1801
weighted avg       0.76      0.75      0.75      1801



In [24]:
# Check if GPU is available
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

Using device: cuda


In [7]:
import random
import torch
import time
from transformers import LongformerForSequenceClassification, LongformerTokenizer

torch.cuda.empty_cache()


# Set random seed for reproducibility
random.seed(42)

# Load the saved model and tokenizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_path = './longformer_model4'  # Path to your saved model
model = LongformerForSequenceClassification.from_pretrained(model_path).to(device)
tokenizer = LongformerTokenizer.from_pretrained(model_path)

print("Model and tokenizer successfully loaded!")

# Limit to 100 articles from the validation set
val_texts_subset = val_texts[:100]
val_labels_subset = val_labels[:100]

# Tokenize the subset of the validation set
start_tokenization_time = time.time()
inputs = tokenizer(val_texts_subset, return_tensors="pt", truncation=True, padding=True, max_length=512).to(device)
end_tokenization_time = time.time()

# Perform inference
model.eval()
start_inference_time = time.time()
with torch.no_grad():
    outputs = model(**inputs)
end_inference_time = time.time()

# Get the predicted class for each input
logits = outputs.logits
predicted_classes = torch.argmax(logits, dim=1).tolist()

# Print results for the validation set subset
label_mapping = {0: "Positive", 1: "Neutral", 2: "Negative"}
correct_predictions = 0

for i, (text, true_label, predicted_label) in enumerate(zip(val_texts_subset, val_labels_subset, predicted_classes)):
    print(f"Text {i+1}: {text}")
    print(f"True Label: {label_mapping[true_label]}, Predicted Label: {label_mapping[predicted_label]}\n")
    if true_label == predicted_label:
        correct_predictions += 1

# Calculate overall accuracy
accuracy = correct_predictions / len(val_labels_subset)
print(f"Accuracy on the validation set: {accuracy:.4f}")

# Calculate timing metrics
tokenization_time = end_tokenization_time - start_tokenization_time
inference_time = end_inference_time - start_inference_time
print(f"Time taken for tokenization: {tokenization_time:.4f} seconds")
print(f"Time taken for inference: {inference_time:.4f} seconds")
print(f"Total time taken: {tokenization_time + inference_time:.4f} seconds")

Model and tokenizer successfully loaded!


Initializing global attention on CLS token...


Text 1: Kompas.tvKONA Electric dihadirkan Hyundai saat Festival Merdeka Berkendara di Palembang, Sumatera Selatan 24 Agustus Hingga 25 Agustus 2024, serta tujuh kota lain di Indonesia PALEMBANG, KOMPAS.TV - PT Hyundai Motors Indonesia terus memperluas jangkauan bisnisnya, khususnya dalam penjualan mobil listrik di Sumatra Selatan . Model terbaru yang diluncurkan adalah KONA Electric generasi terbaru, sebuah mobil listrik dengan berbagai fitur canggih untuk meningkatkan kenyamanan berkendara.  Alay menambahkan bahwa perkembangan ekonomi dan bisnis di Sumsel, terutama di Palembang, menjadikannya salah satu wilayah dengan pembelian mobil listrik tertinggi di Sumatra."Kontribusi pendapatan daerah dan masyarakat Sumsel menjadi alasan kami menargetkan penjualan di sini," tuturnya. "Kami sangat bangga memperkenalkan all-new KONA Electric kepada masyarakat Palembang. Hyundai berkomitmen untuk menghadirkan inovasi terkini demi memberikan pengalaman mobilitas yang baru dan segar," kata Fransiscu

In [ ]:
import random
import torch
import time
from transformers import LongformerForSequenceClassification, LongformerTokenizer

torch.cuda.empty_cache()


# Set random seed for reproducibility
random.seed(42)

# Load the saved model and tokenizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_path = './longformer_model5'  # Path to your saved model
model = LongformerForSequenceClassification.from_pretrained(model_path).to(device)
tokenizer = LongformerTokenizer.from_pretrained(model_path)

print("Model and tokenizer successfully loaded!")

# Limit to 100 articles from the validation set
val_texts_subset = val_texts[:100]
val_labels_subset = val_labels[:100]

# Tokenize the subset of the validation set
start_tokenization_time = time.time()
inputs = tokenizer(val_texts_subset, return_tensors="pt", truncation=True, padding=True, max_length=512).to(device)
end_tokenization_time = time.time()

# Perform inference
model.eval()
start_inference_time = time.time()
with torch.no_grad():
    outputs = model(**inputs)
end_inference_time = time.time()

# Get the predicted class for each input
logits = outputs.logits
predicted_classes = torch.argmax(logits, dim=1).tolist()

# Print results for the validation set subset
label_mapping = {0: "Positive", 1: "Neutral", 2: "Negative"}
correct_predictions = 0

for i, (text, true_label, predicted_label) in enumerate(zip(val_texts_subset, val_labels_subset, predicted_classes)):
    print(f"Text {i+1}: {text}")
    print(f"True Label: {label_mapping[true_label]}, Predicted Label: {label_mapping[predicted_label]}\n")
    if true_label == predicted_label:
        correct_predictions += 1

# Calculate overall accuracy
accuracy = correct_predictions / len(val_labels_subset)
print(f"Accuracy on the validation set: {accuracy:.4f}")

# Calculate timing metrics
tokenization_time = end_tokenization_time - start_tokenization_time
inference_time = end_inference_time - start_inference_time
print(f"Time taken for tokenization: {tokenization_time:.4f} seconds")
print(f"Time taken for inference: {inference_time:.4f} seconds")
print(f"Total time taken: {tokenization_time + inference_time:.4f} seconds")

Model and tokenizer successfully loaded!
Text 1: Kompas.tvKONA Electric dihadirkan Hyundai saat Festival Merdeka Berkendara di Palembang, Sumatera Selatan 24 Agustus Hingga 25 Agustus 2024, serta tujuh kota lain di Indonesia PALEMBANG, KOMPAS.TV - PT Hyundai Motors Indonesia terus memperluas jangkauan bisnisnya, khususnya dalam penjualan mobil listrik di Sumatra Selatan . Model terbaru yang diluncurkan adalah KONA Electric generasi terbaru, sebuah mobil listrik dengan berbagai fitur canggih untuk meningkatkan kenyamanan berkendara.  Alay menambahkan bahwa perkembangan ekonomi dan bisnis di Sumsel, terutama di Palembang, menjadikannya salah satu wilayah dengan pembelian mobil listrik tertinggi di Sumatra."Kontribusi pendapatan daerah dan masyarakat Sumsel menjadi alasan kami menargetkan penjualan di sini," tuturnya. "Kami sangat bangga memperkenalkan all-new KONA Electric kepada masyarakat Palembang. Hyundai berkomitmen untuk menghadirkan inovasi terkini demi memberikan pengalaman mobil

: 

In [12]:
import random
import torch
import time
from transformers import LongformerForSequenceClassification, LongformerTokenizer

torch.cuda.empty_cache()


# Set random seed for reproducibility
random.seed(42)

# Load the saved model and tokenizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_path = './longformer_model2'  # Path to your saved model
model = LongformerForSequenceClassification.from_pretrained(model_path).to(device)
tokenizer = LongformerTokenizer.from_pretrained(model_path)

print("Model and tokenizer successfully loaded!")

# Limit to 100 articles from the validation set
val_texts_subset = val_texts[:100]
val_labels_subset = val_labels[:100]

# Tokenize the subset of the validation set
start_tokenization_time = time.time()
inputs = tokenizer(val_texts_subset, return_tensors="pt", truncation=True, padding=True, max_length=512).to(device)
end_tokenization_time = time.time()

# Perform inference
model.eval()
start_inference_time = time.time()
with torch.no_grad():
    outputs = model(**inputs)
end_inference_time = time.time()

# Get the predicted class for each input
logits = outputs.logits
predicted_classes = torch.argmax(logits, dim=1).tolist()

# Print results for the validation set subset
label_mapping = {0: "Positive", 1: "Neutral", 2: "Negative"}
correct_predictions = 0

for i, (text, true_label, predicted_label) in enumerate(zip(val_texts_subset, val_labels_subset, predicted_classes)):
    print(f"Text {i+1}: {text}")
    print(f"True Label: {label_mapping[true_label]}, Predicted Label: {label_mapping[predicted_label]}\n")
    if true_label == predicted_label:
        correct_predictions += 1

# Calculate overall accuracy
accuracy = correct_predictions / len(val_labels_subset)
print(f"Accuracy on the validation set: {accuracy:.4f}")

# Calculate timing metrics
tokenization_time = end_tokenization_time - start_tokenization_time
inference_time = end_inference_time - start_inference_time
print(f"Time taken for tokenization: {tokenization_time:.4f} seconds")
print(f"Time taken for inference: {inference_time:.4f} seconds")
print(f"Total time taken: {tokenization_time + inference_time:.4f} seconds")

Model and tokenizer successfully loaded!
Text 1: Kompas.tvKONA Electric dihadirkan Hyundai saat Festival Merdeka Berkendara di Palembang, Sumatera Selatan 24 Agustus Hingga 25 Agustus 2024, serta tujuh kota lain di Indonesia PALEMBANG, KOMPAS.TV - PT Hyundai Motors Indonesia terus memperluas jangkauan bisnisnya, khususnya dalam penjualan mobil listrik di Sumatra Selatan . Model terbaru yang diluncurkan adalah KONA Electric generasi terbaru, sebuah mobil listrik dengan berbagai fitur canggih untuk meningkatkan kenyamanan berkendara.  Alay menambahkan bahwa perkembangan ekonomi dan bisnis di Sumsel, terutama di Palembang, menjadikannya salah satu wilayah dengan pembelian mobil listrik tertinggi di Sumatra."Kontribusi pendapatan daerah dan masyarakat Sumsel menjadi alasan kami menargetkan penjualan di sini," tuturnya. "Kami sangat bangga memperkenalkan all-new KONA Electric kepada masyarakat Palembang. Hyundai berkomitmen untuk menghadirkan inovasi terkini demi memberikan pengalaman mobil

In [13]:
import random
import torch
import time
from transformers import LongformerForSequenceClassification, LongformerTokenizer

torch.cuda.empty_cache()


# Set random seed for reproducibility
random.seed(42)

# Load the saved model and tokenizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_path = './longformer_model3'  # Path to your saved model
model = LongformerForSequenceClassification.from_pretrained(model_path).to(device)
tokenizer = LongformerTokenizer.from_pretrained(model_path)

print("Model and tokenizer successfully loaded!")

# Limit to 100 articles from the validation set
val_texts_subset = val_texts[:100]
val_labels_subset = val_labels[:100]

# Tokenize the subset of the validation set
start_tokenization_time = time.time()
inputs = tokenizer(val_texts_subset, return_tensors="pt", truncation=True, padding=True, max_length=512).to(device)
end_tokenization_time = time.time()

# Perform inference
model.eval()
start_inference_time = time.time()
with torch.no_grad():
    outputs = model(**inputs)
end_inference_time = time.time()

# Get the predicted class for each input
logits = outputs.logits
predicted_classes = torch.argmax(logits, dim=1).tolist()

# Print results for the validation set subset
label_mapping = {0: "Positive", 1: "Neutral", 2: "Negative"}
correct_predictions = 0

for i, (text, true_label, predicted_label) in enumerate(zip(val_texts_subset, val_labels_subset, predicted_classes)):
    print(f"Text {i+1}: {text}")
    print(f"True Label: {label_mapping[true_label]}, Predicted Label: {label_mapping[predicted_label]}\n")
    if true_label == predicted_label:
        correct_predictions += 1

# Calculate overall accuracy
accuracy = correct_predictions / len(val_labels_subset)
print(f"Accuracy on the validation set: {accuracy:.4f}")

# Calculate timing metrics
tokenization_time = end_tokenization_time - start_tokenization_time
inference_time = end_inference_time - start_inference_time
print(f"Time taken for tokenization: {tokenization_time:.4f} seconds")
print(f"Time taken for inference: {inference_time:.4f} seconds")
print(f"Total time taken: {tokenization_time + inference_time:.4f} seconds")

Model and tokenizer successfully loaded!
Text 1: Kompas.tvKONA Electric dihadirkan Hyundai saat Festival Merdeka Berkendara di Palembang, Sumatera Selatan 24 Agustus Hingga 25 Agustus 2024, serta tujuh kota lain di Indonesia PALEMBANG, KOMPAS.TV - PT Hyundai Motors Indonesia terus memperluas jangkauan bisnisnya, khususnya dalam penjualan mobil listrik di Sumatra Selatan . Model terbaru yang diluncurkan adalah KONA Electric generasi terbaru, sebuah mobil listrik dengan berbagai fitur canggih untuk meningkatkan kenyamanan berkendara.  Alay menambahkan bahwa perkembangan ekonomi dan bisnis di Sumsel, terutama di Palembang, menjadikannya salah satu wilayah dengan pembelian mobil listrik tertinggi di Sumatra."Kontribusi pendapatan daerah dan masyarakat Sumsel menjadi alasan kami menargetkan penjualan di sini," tuturnya. "Kami sangat bangga memperkenalkan all-new KONA Electric kepada masyarakat Palembang. Hyundai berkomitmen untuk menghadirkan inovasi terkini demi memberikan pengalaman mobil

In [14]:
import random
import torch
import time
from transformers import LongformerForSequenceClassification, LongformerTokenizer

torch.cuda.empty_cache()


# Set random seed for reproducibility
random.seed(42)

# Load the saved model and tokenizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_path = './longformer_model6'  # Path to your saved model
model = LongformerForSequenceClassification.from_pretrained(model_path).to(device)
tokenizer = LongformerTokenizer.from_pretrained(model_path)

print("Model and tokenizer successfully loaded!")

# Limit to 100 articles from the validation set
val_texts_subset = val_texts[:100]
val_labels_subset = val_labels[:100]

# Tokenize the subset of the validation set
start_tokenization_time = time.time()
inputs = tokenizer(val_texts_subset, return_tensors="pt", truncation=True, padding=True, max_length=512).to(device)
end_tokenization_time = time.time()

# Perform inference
model.eval()
start_inference_time = time.time()
with torch.no_grad():
    outputs = model(**inputs)
end_inference_time = time.time()

# Get the predicted class for each input
logits = outputs.logits
predicted_classes = torch.argmax(logits, dim=1).tolist()

# Print results for the validation set subset
label_mapping = {0: "Positive", 1: "Neutral", 2: "Negative"}
correct_predictions = 0

for i, (text, true_label, predicted_label) in enumerate(zip(val_texts_subset, val_labels_subset, predicted_classes)):
    print(f"Text {i+1}: {text}")
    print(f"True Label: {label_mapping[true_label]}, Predicted Label: {label_mapping[predicted_label]}\n")
    if true_label == predicted_label:
        correct_predictions += 1

# Calculate overall accuracy
accuracy = correct_predictions / len(val_labels_subset)
print(f"Accuracy on the validation set: {accuracy:.4f}")

# Calculate timing metrics
tokenization_time = end_tokenization_time - start_tokenization_time
inference_time = end_inference_time - start_inference_time
print(f"Time taken for tokenization: {tokenization_time:.4f} seconds")
print(f"Time taken for inference: {inference_time:.4f} seconds")
print(f"Total time taken: {tokenization_time + inference_time:.4f} seconds")

Model and tokenizer successfully loaded!
Text 1: Kompas.tvKONA Electric dihadirkan Hyundai saat Festival Merdeka Berkendara di Palembang, Sumatera Selatan 24 Agustus Hingga 25 Agustus 2024, serta tujuh kota lain di Indonesia PALEMBANG, KOMPAS.TV - PT Hyundai Motors Indonesia terus memperluas jangkauan bisnisnya, khususnya dalam penjualan mobil listrik di Sumatra Selatan . Model terbaru yang diluncurkan adalah KONA Electric generasi terbaru, sebuah mobil listrik dengan berbagai fitur canggih untuk meningkatkan kenyamanan berkendara.  Alay menambahkan bahwa perkembangan ekonomi dan bisnis di Sumsel, terutama di Palembang, menjadikannya salah satu wilayah dengan pembelian mobil listrik tertinggi di Sumatra."Kontribusi pendapatan daerah dan masyarakat Sumsel menjadi alasan kami menargetkan penjualan di sini," tuturnya. "Kami sangat bangga memperkenalkan all-new KONA Electric kepada masyarakat Palembang. Hyundai berkomitmen untuk menghadirkan inovasi terkini demi memberikan pengalaman mobil

In [15]:
import random
import torch
import time
from transformers import LongformerForSequenceClassification, LongformerTokenizer

torch.cuda.empty_cache()


# Set random seed for reproducibility
random.seed(42)

# Load the saved model and tokenizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_path = './longformer_model7'  # Path to your saved model
model = LongformerForSequenceClassification.from_pretrained(model_path).to(device)
tokenizer = LongformerTokenizer.from_pretrained(model_path)

print("Model and tokenizer successfully loaded!")

# Limit to 100 articles from the validation set
val_texts_subset = val_texts[:100]
val_labels_subset = val_labels[:100]

# Tokenize the subset of the validation set
start_tokenization_time = time.time()
inputs = tokenizer(val_texts_subset, return_tensors="pt", truncation=True, padding=True, max_length=512).to(device)
end_tokenization_time = time.time()

# Perform inference
model.eval()
start_inference_time = time.time()
with torch.no_grad():
    outputs = model(**inputs)
end_inference_time = time.time()

# Get the predicted class for each input
logits = outputs.logits
predicted_classes = torch.argmax(logits, dim=1).tolist()

# Print results for the validation set subset
label_mapping = {0: "Positive", 1: "Neutral", 2: "Negative"}
correct_predictions = 0

for i, (text, true_label, predicted_label) in enumerate(zip(val_texts_subset, val_labels_subset, predicted_classes)):
    print(f"Text {i+1}: {text}")
    print(f"True Label: {label_mapping[true_label]}, Predicted Label: {label_mapping[predicted_label]}\n")
    if true_label == predicted_label:
        correct_predictions += 1

# Calculate overall accuracy
accuracy = correct_predictions / len(val_labels_subset)
print(f"Accuracy on the validation set: {accuracy:.4f}")

# Calculate timing metrics
tokenization_time = end_tokenization_time - start_tokenization_time
inference_time = end_inference_time - start_inference_time
print(f"Time taken for tokenization: {tokenization_time:.4f} seconds")
print(f"Time taken for inference: {inference_time:.4f} seconds")
print(f"Total time taken: {tokenization_time + inference_time:.4f} seconds")

Model and tokenizer successfully loaded!
Text 1: Kompas.tvKONA Electric dihadirkan Hyundai saat Festival Merdeka Berkendara di Palembang, Sumatera Selatan 24 Agustus Hingga 25 Agustus 2024, serta tujuh kota lain di Indonesia PALEMBANG, KOMPAS.TV - PT Hyundai Motors Indonesia terus memperluas jangkauan bisnisnya, khususnya dalam penjualan mobil listrik di Sumatra Selatan . Model terbaru yang diluncurkan adalah KONA Electric generasi terbaru, sebuah mobil listrik dengan berbagai fitur canggih untuk meningkatkan kenyamanan berkendara.  Alay menambahkan bahwa perkembangan ekonomi dan bisnis di Sumsel, terutama di Palembang, menjadikannya salah satu wilayah dengan pembelian mobil listrik tertinggi di Sumatra."Kontribusi pendapatan daerah dan masyarakat Sumsel menjadi alasan kami menargetkan penjualan di sini," tuturnya. "Kami sangat bangga memperkenalkan all-new KONA Electric kepada masyarakat Palembang. Hyundai berkomitmen untuk menghadirkan inovasi terkini demi memberikan pengalaman mobil

In [16]:
import random
import torch
import time
from transformers import LongformerForSequenceClassification, LongformerTokenizer

torch.cuda.empty_cache()


# Set random seed for reproducibility
random.seed(42)

# Load the saved model and tokenizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_path = './longformer_model8'  # Path to your saved model
model = LongformerForSequenceClassification.from_pretrained(model_path).to(device)
tokenizer = LongformerTokenizer.from_pretrained(model_path)

print("Model and tokenizer successfully loaded!")

# Limit to 100 articles from the validation set
val_texts_subset = val_texts[:100]
val_labels_subset = val_labels[:100]

# Tokenize the subset of the validation set
start_tokenization_time = time.time()
inputs = tokenizer(val_texts_subset, return_tensors="pt", truncation=True, padding=True, max_length=512).to(device)
end_tokenization_time = time.time()

# Perform inference
model.eval()
start_inference_time = time.time()
with torch.no_grad():
    outputs = model(**inputs)
end_inference_time = time.time()

# Get the predicted class for each input
logits = outputs.logits
predicted_classes = torch.argmax(logits, dim=1).tolist()

# Print results for the validation set subset
label_mapping = {0: "Positive", 1: "Neutral", 2: "Negative"}
correct_predictions = 0

for i, (text, true_label, predicted_label) in enumerate(zip(val_texts_subset, val_labels_subset, predicted_classes)):
    print(f"Text {i+1}: {text}")
    print(f"True Label: {label_mapping[true_label]}, Predicted Label: {label_mapping[predicted_label]}\n")
    if true_label == predicted_label:
        correct_predictions += 1

# Calculate overall accuracy
accuracy = correct_predictions / len(val_labels_subset)
print(f"Accuracy on the validation set: {accuracy:.4f}")

# Calculate timing metrics
tokenization_time = end_tokenization_time - start_tokenization_time
inference_time = end_inference_time - start_inference_time
print(f"Time taken for tokenization: {tokenization_time:.4f} seconds")
print(f"Time taken for inference: {inference_time:.4f} seconds")
print(f"Total time taken: {tokenization_time + inference_time:.4f} seconds")

Model and tokenizer successfully loaded!
Text 1: Kompas.tvKONA Electric dihadirkan Hyundai saat Festival Merdeka Berkendara di Palembang, Sumatera Selatan 24 Agustus Hingga 25 Agustus 2024, serta tujuh kota lain di Indonesia PALEMBANG, KOMPAS.TV - PT Hyundai Motors Indonesia terus memperluas jangkauan bisnisnya, khususnya dalam penjualan mobil listrik di Sumatra Selatan . Model terbaru yang diluncurkan adalah KONA Electric generasi terbaru, sebuah mobil listrik dengan berbagai fitur canggih untuk meningkatkan kenyamanan berkendara.  Alay menambahkan bahwa perkembangan ekonomi dan bisnis di Sumsel, terutama di Palembang, menjadikannya salah satu wilayah dengan pembelian mobil listrik tertinggi di Sumatra."Kontribusi pendapatan daerah dan masyarakat Sumsel menjadi alasan kami menargetkan penjualan di sini," tuturnya. "Kami sangat bangga memperkenalkan all-new KONA Electric kepada masyarakat Palembang. Hyundai berkomitmen untuk menghadirkan inovasi terkini demi memberikan pengalaman mobil

: 

In [25]:
# Load the saved model and tokenizer
model = LongformerForSequenceClassification.from_pretrained('./longformer_model4').to(device)
tokenizer = LongformerTokenizer.from_pretrained('./longformer_model4')

In [26]:
# Define label mapping
label_mapping = {0: "Negative", 1: "Neutral", 2: "Positive"}

In [27]:
# Testing new articles
def classify_article(article):
    # Tokenize the input article and move the tensors to the correct device
    inputs = tokenizer(article, return_tensors="pt", truncation=True, padding=True, max_length=512)
    
    # Move tokenized inputs to the correct device (GPU or CPU)
    inputs = {key: value.to(device) for key, value in inputs.items()}
    
    with torch.no_grad():
        # Perform inference using the model
        outputs = model(**inputs)
    
    logits = outputs.logits
    # Get the predicted label (index of the highest logit)
    predicted_label = np.argmax(logits.cpu().numpy(), axis=1)[0]
    
    # Calculate sentiment scores (probabilities)
    sentiment_score = torch.softmax(logits, dim=1).cpu().numpy()[0]
    
    # Map predicted label to sentiment text
    sentiment_result = label_mapping[predicted_label]
    
    return sentiment_result, sentiment_score

In [28]:
# User input for a new article
new_article = input("Your article:")
sentiment_result, sentiment_score = classify_article(new_article)
print(f"The sentiment of the article is: {sentiment_result}")
print(f"Sentiment scores: {sentiment_score}")


Input ids are automatically padded to be a multiple of `config.attention_window`: 512


The sentiment of the article is: Negative
Sentiment scores: [0.5269819  0.39744824 0.07556986]


In [29]:
from transformers import LongformerTokenizer, EncoderDecoderModel

# Load summarization model and tokenizer
summarization_model = EncoderDecoderModel.from_pretrained("patrickvonplaten/longformer2roberta-cnn_dailymail-fp16")
summarization_tokenizer = LongformerTokenizer.from_pretrained("allenai/longformer-base-4096")

You are using a model of type encoder_decoder to instantiate a model of type encoder-decoder. This is not supported for all configurations of models and can yield errors.
EncoderDecoderModel has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to 

In [30]:
# Function to summarize text
def summarize_text(text):
    input_ids = summarization_tokenizer(text, return_tensors="pt", max_length=4096, truncation=True).input_ids
    output_ids = summarization_model.generate(input_ids, max_length=150, min_length=30, length_penalty=2.0, num_beams=4, early_stopping=True)
    return summarization_tokenizer.decode(output_ids[0], skip_special_tokens=True)


In [31]:
# User input for text summarization
article_to_summarize = input("Your article:")

In [32]:
# Ensure model is moved to the device
summarization_model = summarization_model.to(device)

# Tokenize and summarize
input_ids = summarization_tokenizer(article_to_summarize, return_tensors="pt").input_ids.to(device)
output_ids = summarization_model.generate(input_ids)

# Get the summary from the output tokens
summary = summarization_tokenizer.decode(output_ids[0], skip_special_tokens=True)

# Print summary
print("Summary of the article:")
print(summary)

Summary of the article:
The former U.S. president is among the most successful in the world .
He is also the subject of a series of interviews with the media .
The interview was conducted by the University of New York .
It was conducted in response to a request from the public .


In [33]:
from transformers import BartForConditionalGeneration, BartTokenizer

summarization_model = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn").to(device)
tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")

# Article text to summarize
article_to_summarize = """
Jakarta, VIVA – Pembayaran kredit sepeda motor yang seringkali menjadi beban bagi sebagian besar masyarakat, kini 
dapat diatasi melalui program Hari Pembayaran Berbayar atau Harcilnas 2024 yang diselenggarakan oleh PT Adira Dinamika 
Multi Finance Tbk (Adira Finance). Program ini memungkinkan nasabah melunasi kredit sepeda motor atau kredit lainnya secara 
penuh hanya dengan menukarkan poin yang terkumpul. Pada Harcilnas periode pertama tahun ini, ada 12 nasabah yang berhasil 
melunasi seluruh tagihannya.
"""

# Use "summarize:" prefix
input_text = f"summarize: {article_to_summarize}"

# Tokenize the input text
input_ids = tokenizer(input_text, return_tensors="pt", truncation=True, padding=True, max_length=512).input_ids.to(device)

# Generate the summary with BART's optimized settings
output_ids = summarization_model.generate(
    input_ids,
    max_length=150,            # Shorter summary length
    num_beams=4,               # Beam search for better quality
    early_stopping=True,
)

# Decode the generated summary output
summary = tokenizer.decode(output_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)
print("Summary of the article:")
print(summary)


c:\Users\rache\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\models\bart\modeling_bart.py:497: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


Summary of the article:
Pembayaran kredit sepeda motor yang seringkali menjadi beban bagi sebagian besar masyarakat. Pada Harcilnas periode pertama tahun ini, ada 12 nasabah yang berhasil melunasi seluruh tagihannya.


In [34]:
# Export results to CSV
import csv

def export_results(sentiment_results, summaries):
    with open('results.csv', mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(['Article', 'Sentiment', 'Sentiment Scores', 'Summary'])
        for article, sentiment, scores, summary in zip([new_article, article_to_summarize], 
                                                        [sentiment_result, 'N/A'], 
                                                        [sentiment_score, 'N/A'], 
                                                        [summary, 'N/A']):
            writer.writerow([article, sentiment, scores.tolist() if isinstance(scores, np.ndarray) else scores, summary])

In [35]:
# Call the export function
export_results(sentiment_results=[(new_article, sentiment_result, sentiment_score)], 
               summaries=[summary])
print("Results exported to 'results.csv'.")

UnboundLocalError: cannot access local variable 'summary' where it is not associated with a value